# Lib


In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torchvision.utils import save_image

from tqdm import tqdm
from models.srcnn import *
from models.vdsr import *

from models.sr_model import *
from models.edrn import *
from models.vdsr import *
from models.srresnet_ import *
from models.sr_model import *
from models.vdsr import *
from models.utils import *
from models.srcnn import *
from models.edsr import *
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

# Model

In [2]:
# edsr_srcnn = EDSR_srcnnfy()
edsr = EDSR().to(device)
edrn_canny = EDRN(use_canny=True).to(device)
edrn_sobel = EDRN(use_sobel=True).to(device)
srresnet = SRResNet().to(device)
vdsr = VDSR().to(device)
srcnn = SRCNN().to(device)


In [3]:

edsr.load_state_dict(torch.load('best_edsr.pth', map_location=device))
edrn_sobel.load_state_dict(torch.load('weight_pcb/best_sobel_srx4_model.pth', map_location=device))
edrn_canny.load_state_dict(torch.load('weight_pcb/best_canny_srx4_model.pth', map_location=device))
srresnet.load_state_dict(torch.load('best_srresnet.pth', map_location=device))
vdsr.load_state_dict(torch.load('weight_pcb/best_vdsr.pth', map_location=device))
srcnn.load_state_dict(torch.load('weight_pcb/best_srcnn.pth', map_location=device))


/tmp/ipykernel_149750/957710577.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  edsr.load_state_dict(torch.load('best_edsr.pth', map_location=device))
/tmp/ipykernel_149

<All keys matched successfully>

# Evaluation


In [6]:
from torchmetrics import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure

def calculate_metrics(img1, img2, max_pixel_value=1.0):
    psnr = PeakSignalNoiseRatio(data_range=1.0)
    ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
    psnr_value = psnr(img1, img2)
    ssim_value = ssim(img1, img2)
    return psnr_value, ssim_value

In [7]:
class ImageDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, type =False):
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir
        self.type =type

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_image = cv2.imread(os.path.join(self.lr_dir, self.lr_files[idx]))
        hr_image = cv2.imread(os.path.join(self.hr_dir, self.hr_files[idx]))
        lr_image = cv2.cvtColor(lr_image, cv2.COLOR_BGR2RGB)
        hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        if self.type:
            height, width = lr_image.shape[:2]
            scale_factor = 4
            # lr_height, lr_width = height // 2, width // 2
            hr_height, hr_width = height * 4, width * 4

            # # Rescale ảnh LR lên 4 lần
            # lr_image = cv2.resize(lr_image, (lr_width, lr_height), interpolation=cv2.INTER_CUBIC)
            lr_image = cv2.resize(lr_image, (hr_width, hr_height), interpolation=cv2.INTER_CUBIC)
            
        # hr_image = cv2.resize(hr_image, (hr_width, hr_height), interpolation=cv2.INTER_CUBIC)

        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor()
        ])
        
        lr_image = transform(lr_image)
        hr_image = transform(hr_image)
        return lr_image, hr_image
    

In [8]:
import time
sub = 'DIV2K'
valid_lr_dir = f'benchmark/{sub}/LR'
valid_hr_dir = f'benchmark/{sub}/HR'
output_image_dir = f'benchmark/output/{sub}'
valid_dataset = ImageDataset(valid_lr_dir, valid_hr_dir)
valid_dataset_2 = ImageDataset(valid_lr_dir, valid_hr_dir, type = True)
valid_loader = DataLoader(valid_dataset)
valid_loader_2 = DataLoader(valid_dataset_2)
log_file = open('subtracttion.txt', 'a')
log_file.write(f'{sub}\n')
# Đo thời gian xử lý trung bình cho từng mô hình
sobel_total_time = 0
canny_total_time = 0
edsr_total_time = 0
srresnet_total_time = 0
vdsr_total_time = 0
srcnn_total_time = 0
bicubic_total_time = 0

edrn_sobel.eval()
edrn_canny.eval()
srresnet.eval()
srcnn.eval()
edsr.eval()
vdsr.eval()

val_psnr_values_bicubic = 0
val_psnr_values_sobel = 0
val_psnr_values_canny = 0
val_psnr_values_edsr = 0
val_psnr_values_srresnet = 0
val_psnr_values_vdsr = 0
val_psnr_values_srcnn = 0

val_ssim_values_sobel = 0
val_ssim_values_canny = 0
val_ssim_values_edsr = 0
val_ssim_values_srresnet = 0
val_ssim_values_vdsr = 0
val_ssim_values_srcnn = 0
val_ssim_values_bicubic = 0

torch.cuda.empty_cache()
with torch.no_grad():  # Không tính toán gradient trong quá trình validation
    for (lr_images, hr_images) in tqdm(valid_loader_2, unit='batch'):
        # VDSR validation
        start_time = time.time()
        outputs_vdsr = vdsr(lr_images)
        vdsr_total_time += time.time() - start_time
        psnr_vdsr, ssim_vdsr = calculate_metrics(outputs_vdsr, hr_images)

        # SRCNN validation
        start_time = time.time()
        outputs_srcnn = srcnn(lr_images)
        srcnn_total_time += time.time() - start_time
        psnr_srcnn, ssim_srcnn = calculate_metrics(outputs_srcnn, hr_images)

        # Cập nhật PSNR và SSIM cho VDSR và SRCNN
        val_psnr_values_vdsr += psnr_vdsr
        val_psnr_values_srcnn += psnr_srcnn

        val_ssim_values_vdsr += ssim_vdsr
        val_ssim_values_srcnn += ssim_srcnn

    # Tính PSNR và SSIM trung bình cho VDSR và SRCNN
    val_average_psnr_vdsr = val_psnr_values_vdsr / len(valid_loader_2)
    val_average_psnr_srcnn = val_psnr_values_srcnn / len(valid_loader_2)

    val_average_ssim_vdsr = val_ssim_values_vdsr / len(valid_loader_2)
    val_average_ssim_srcnn = val_ssim_values_srcnn / len(valid_loader_2)

    # Thời gian xử lý trung bình cho VDSR và SRCNN
    avg_time_vdsr = vdsr_total_time / len(valid_loader_2)
    avg_time_srcnn = srcnn_total_time / len(valid_loader_2)

    # Ghi kết quả vào log_file
    
torch.cuda.empty_cache()
with torch.no_grad():  # Không tính toán gradient trong quá trình validation
    for (lr_images, hr_images) in tqdm(valid_loader, unit='batch'):
        lr_images = lr_images.to(device)
        hr_images = hr_images.to(device)

        # Sobel SR validation (không tính loss, chỉ tính PSNR và SSIM)
        start_time = time.time()  # Đo thời gian bắt đầu
        outputs_sobel = edrn_sobel(lr_images)
        sobel_total_time += time.time() - start_time  # Tính thời gian xử lý
        psnr_sobel, ssim_sobel = calculate_metrics(outputs_sobel, hr_images)

        #bicubic
        start_time = time.time()  # Đo thời gian bắt đầu
        outputs_bicubic = F.interpolate(lr_images, scale_factor=4, mode='bilinear', align_corners=False)
        bicubic_total_time += time.time() - start_time  # Tính thời gian xử lý bicubic
        psnr_bicubic, ssim_bicubic = calculate_metrics(outputs_bicubic, hr_images)

        # Canny SR validation
        start_time = time.time()
        outputs_canny = edrn_canny(lr_images)
        canny_total_time += time.time() - start_time
        psnr_canny, ssim_canny = calculate_metrics(outputs_canny, hr_images)

        # EDSR SR validation
        start_time = time.time()
        outputs_edsr = edsr(lr_images)
        edsr_total_time += time.time() - start_time
        psnr_edsr, ssim_edsr = calculate_metrics(outputs_edsr, hr_images)

        # SRResNet SR validation
        start_time = time.time()
        outputs_srresnet = srresnet(lr_images)
        srresnet_total_time += time.time() - start_time
        psnr_srresnet, ssim_srresnet = calculate_metrics(outputs_srresnet, hr_images)

        # Cập nhật PSNR và SSIM
        val_psnr_values_sobel += psnr_sobel
        val_psnr_values_canny += psnr_canny
        val_psnr_values_edsr += psnr_edsr
        val_psnr_values_srresnet += psnr_srresnet
        val_psnr_values_bicubic += psnr_bicubic
        
        val_ssim_values_sobel += ssim_sobel
        val_ssim_values_canny += ssim_canny
        val_ssim_values_edsr += ssim_edsr
        val_ssim_values_srresnet += ssim_srresnet
        val_ssim_values_bicubic += ssim_bicubic
    # Tính PSNR và SSIM trung bình
    val_average_psnr_sobel = val_psnr_values_sobel / len(valid_loader)
    val_average_psnr_canny = val_psnr_values_canny / len(valid_loader)
    val_average_psnr_edsr = val_psnr_values_edsr / len(valid_loader)
    val_average_psnr_srresnet = val_psnr_values_srresnet / len(valid_loader)
    val_average_psnr_bicubic = val_psnr_values_bicubic / len(valid_loader)
    
    val_average_ssim_sobel = val_ssim_values_sobel / len(valid_loader)
    val_average_ssim_canny = val_ssim_values_canny / len(valid_loader)
    val_average_ssim_edsr = val_ssim_values_edsr / len(valid_loader)
    val_average_ssim_srresnet = val_ssim_values_srresnet / len(valid_loader)
    val_average_ssim_bicubic = val_ssim_values_bicubic / len(valid_loader)

    # Thời gian xử lý trung bình
    avg_time_sobel = sobel_total_time / len(valid_loader)
    avg_time_canny = canny_total_time / len(valid_loader)
    avg_time_edsr = edsr_total_time / len(valid_loader)
    avg_time_srresnet = srresnet_total_time / len(valid_loader)
    avg_time_bicubic = bicubic_total_time / len(valid_loader)
    

    # Thời gian xử lý trung bình cho Bicubic
    
    temp_psnr = val_average_psnr_bicubic
    temp_ssim = val_average_ssim_bicubic
    # Ghi kết quả vào log_file
    # Validation cho VDSR và SRCNN
log_file.write(f'Bicubic:  SSIM / PSNR {val_average_ssim_bicubic:.4f} / {val_average_psnr_bicubic:.2f}, Time {avg_time_bicubic:.4f}s\n')
log_file.write(f'VDSR:  SSIM / PSNR {val_average_ssim_vdsr:.4f} / {val_average_psnr_vdsr:.2f}, Time {avg_time_vdsr:.4f}s\n')
log_file.write(f'SRCNN:  SSIM / PSNR {val_average_ssim_srcnn:.4f} / {val_average_psnr_srcnn:.2f}, Time {avg_time_srcnn:.4f}s\n')    
log_file.write(f'SRResNet:  SSIM / PSNR {val_average_ssim_srresnet:.4f} / {val_average_psnr_srresnet:.2f}, Time {avg_time_srresnet:.4f}s\n')
log_file.write(f'EDSR:  SSIM / PSNR {val_average_ssim_edsr:.4f} / {val_average_psnr_edsr:.2f}, Time {avg_time_edsr:.4f}s\n')
log_file.write(f'Sobel:  SSIM / PSNR {val_average_ssim_sobel:.4f} / {val_average_psnr_sobel:.2f}, Time {avg_time_sobel:.4f}s\n')
log_file.write(f'Canny:  SSIM / PSNR {val_average_ssim_canny:.4f} / {val_average_psnr_canny:.2f}, Time {avg_time_canny:.4f}s\n')

log_file.flush()
log_file.close


FileNotFoundError: [Errno 2] No such file or directory: 'benchmark/DIV2K/LR'

In [9]:
import random
import time
from PIL import ImageDraw
transform = transforms.ToTensor()

sub = 'pcb'
box = 80
valid_lr_dir = f'benchmark/{sub}/LR'
valid_hr_dir = f'benchmark/{sub}/HR'
output_image_dir = f'benchmark/output/{sub}'
os.makedirs(output_image_dir, exist_ok=True)
lr_image_dir = valid_lr_dir
hr_image_dir = valid_hr_dir
# index = random.randint(0, len(os.listdir(lr_image_dir))-1)
# file = '0875.png'
# lr_image_file = file
# hr_image_file = file
with torch.no_grad():
    for (lr_image_file, hr_image_file) in zip(sorted(os.listdir(lr_image_dir)), sorted(os.listdir(hr_image_dir))):
        # Đường dẫn đến ảnh

        lr_image_path = os.path.join(lr_image_dir, lr_image_file)
        hr_image_path = os.path.join(hr_image_dir, hr_image_file)
        ####################################
        img = cv2.imread(hr_image_path)  # Thay bằng đường dẫn ảnh của bạn
        cv2.namedWindow("Image")
        cv2.imshow("Image", img)
        drawing = False  # True khi đang nhấn chuột
        top_left_corner = [0,0]
        xy = [0, 0, 0, 0]
        def draw_rectangle(event, x, y, flags, param):
            global drawing, top_left_corner, img, xy
            
            # Khi nhấn chuột trái, bắt đầu vẽ
            if event == cv2.EVENT_LBUTTONDOWN:
                drawing = True
                top_left_corner = [x, y]   # Lưu tọa độ của điểm bắt đầu
            
            # Khi thả chuột trái, kết thúc vẽ
            elif event == cv2.EVENT_LBUTTONUP:
                drawing = False
                
                xy[0] = top_left_corner[0]
                xy[1] = top_left_corner[1]
                xy[2] = top_left_corner[0] + box # type: ignore
                xy[3] = top_left_corner[1] + box
                
                cv2.destroyAllWindows()
                # Đóng cửa sổ ảnh sau khi vẽ xong
        # Đặt callback cho sự kiện chuột
        cv2.setMouseCallback("Image", draw_rectangle)

        # Hiển thị ảnh và đợi cho đến khi bounding box được vẽ
        cv2.waitKey(0)
        ####################################
        edsr_path = os.path.join(output_image_dir, lr_image_file[:-4] + '_edsr.jpg')
        edrn_sobel_path = os.path.join(output_image_dir, lr_image_file[:-4] + '_edrn_sobel.jpg')
        edrn_canny_path = os.path.join(output_image_dir, lr_image_file[:-4] + '_edrn_canny.jpg')
        bicubic_path = os.path.join(output_image_dir, lr_image_file[:-4] + '_bicubic.jpg')
        srresnet_path = os.path.join(output_image_dir, lr_image_file[:-4] + '_srresnet.jpg')
        srcnn_path = os.path.join(output_image_dir, lr_image_file[:-4] + '_srcnn.jpg')
        vdsr_path = os.path.join(output_image_dir, lr_image_file[:-4] + '_vdsr.jpg')

        hr_path = os.path.join(output_image_dir, hr_image_file)
        # Tải và chuyển đổi ảnh
        lr_image = Image.open(lr_image_path)
        hr_image = Image.open(hr_image_path)
        w, h = hr_image.size
        bicubic = lr_image.resize((w, h))
        lr_image_copy = lr_image.copy().convert('RGB')
        hr_image_copy = hr_image.copy()
        # lr_image_1dms = torch.Tensor(lr_image_1dms)
        lr_image = transform(lr_image).unsqueeze(0).to(device)  # Thêm batch dimension và chuyển sang CPU
        hr_image = transform(hr_image).unsqueeze(0).to(device)  # Thêm batch dimension và chuyển sang CPU
        bicubic_ = transform(bicubic).unsqueeze(0).to(device)
        # print(type(lr_image_1dms))
        
        
        
        # bicubic = lr_image_copy.resize((600, 600), resample=Image.BICUBIC) # type: ignore
        # bicubic_ = transform(bicubic).unsqueeze(0).to(device)
        # lr_image_1dms, _ = preprocess(lr_image_copy, device)
        # lr_image_copy, _ = preprocess(lr_image_copy, device)
        # hr_image_copy, _ = preprocess(HR, device)
        # _, ycbcr = preprocess(bicubic, device)
        time_pro = []
        def measure_inference_time(model, input_image, model_name):
            start_time = time.time()
            output = model(input_image)
            end_time = time.time()
            
            inference_time = end_time - start_time
            time_pro.append(inference_time)
            return output
        
        # Dự đoán
        output_edsr = edsr(lr_image)
        output_edrn_sobel = edrn_sobel(lr_image)
        output_edrn_canny = edrn_canny(lr_image)
        output_srresnet = srresnet(lr_image)
        output_srcnn = srcnn(bicubic_)
        output_vdsr = vdsr(bicubic_)
        models = ['EDSR', 'EDRN Sobel', 'EDRN Canny', 'SRResNet', 'SRCNN', 'VDSR']
        psnr_value = []  # Lưu giá trị PSNR cho mỗi mô hình
        ssim_value = []
        # Tính PSNR cho từng mô hình (giả sử output của các mô hình đã có)
        for model_output in [output_edsr, output_edrn_sobel, output_edrn_canny, output_srresnet, output_srcnn, output_vdsr]:
            # Tính PSNR và thêm vào danh sách
            psnr_value.append(calculate_metrics(model_output, hr_image)[0])
            ssim_value.append(calculate_metrics(model_output, hr_image)[1])
        # Ghi PSNR cho từng mô hình vào file
        with open(f"{output_image_dir}/results.txt", "a") as psnr_file:
            psnr_file.write(f"HR Image: {hr_image_file}\n")
            for i, model_name in enumerate(models):
                psnr_file.write(f"{model_name} PSNR/SSIM: {psnr_value[i]:.2f} dB/ {ssim_value[i]:.4f}\n")
            psnr_file.write(f"Bicubic PSNR/SSIM: {calculate_metrics(bicubic_, hr_image)[0]:.2f} dB/ {calculate_metrics(bicubic_, hr_image)[1]:.2f}\n")
            psnr_file.write("\n")  # Dòng trống giữa các lần lặp
            
            
        output_image_edsr = output_edsr.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
        output_image_edsr = transforms.ToPILImage()(output_image_edsr)  # Chuyển tensor thành ảnh PIL
        output_image_edsr = output_image_edsr.crop(xy)
        output_image_edsr.resize((100, 100))
        output_image_edsr.save(edsr_path)  # Lưu ảnh
        
        output_image_edrn_sobel = output_edrn_sobel.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
        output_image_edrn_sobel = transforms.ToPILImage()(output_image_edrn_sobel)  # Chuyển tensor thành ảnh PIL
        output_image_edrn_sobel = output_image_edrn_sobel.crop(xy)
        output_image_edrn_sobel.resize((100, 100))
        output_image_edrn_sobel.save(edrn_sobel_path)  # Lưu ảnh
        
        
        output_image_edrn_canny = output_edrn_canny.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
        output_image_edrn_canny = transforms.ToPILImage()(output_image_edrn_canny)  # Chuyển tensor thành ảnh PIL
        output_image_edrn_canny = output_image_edrn_canny.crop(xy)
        output_image_edrn_canny.resize((100, 100))
        output_image_edrn_canny.save(edrn_canny_path)  # Lưu ảnh
        
        output_image_srcnn = output_srcnn.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
        output_image_srcnn = transforms.ToPILImage()(output_image_srcnn)  # Chuyển tensor thành ảnh PIL
        output_image_srcnn = output_image_srcnn.crop(xy)
        output_image_srcnn.resize((100, 100))
        output_image_srcnn.save(srcnn_path)  # Lưu ảnh
        
        # output_vdsr = output_vdsr.mul(255.0).to(device).numpy().squeeze(0).squeeze(0)
        # output_image_vdsr = np.array([output_vdsr, ycbcr[..., 1], ycbcr[..., 2]]).transpose([1, 2, 0])
        # output_image_vdsr = np.clip(convert_ycbcr_to_rgb(output_image_vdsr ), 0.0, 255.0).astype(np.uint8)
        # output_image_vdsr = Image.fromarray(output_image_vdsr )
        # output_image_vdsr = output_image_vdsr.crop(xy)
        # output_image_vdsr .save(vdsr_path) 
        
        output_image_vdsr = output_vdsr.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
        output_image_vdsr = transforms.ToPILImage()(output_image_vdsr)  # Chuyển tensor thành ảnh PIL
        output_image_vdsr = output_image_vdsr.crop(xy)
        output_image_vdsr.resize((100, 100))
        output_image_vdsr.save(vdsr_path)  # Lưu ản
        
        output_image_srresnet = output_srresnet.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
        output_image_srresnet = transforms.ToPILImage()(output_image_srresnet)  # Chuyển tensor thành ảnh PIL
        output_image_srresnet = output_image_srresnet.crop(xy)
        output_image_srresnet.resize((100, 100))
        output_image_srresnet.save(srresnet_path)  # Lưu ảnh
        
        # bicubic = lr_image.resize((600, 600))
        bicubic = bicubic.crop(xy)
        bicubic.resize((100, 100))
        bicubic.save(bicubic_path)
        
        hr_image_crop = hr_image_copy.crop(xy)
        hr_image_crop.resize((100, 100))
        hr_image_crop.save(hr_path)
        
        draw = ImageDraw.Draw(hr_image_copy)
        draw.rectangle(xy, outline="red", width=3)
        hr_image_copy = hr_image_copy.resize((600, 600))
        hr_image_copy.save(f"{output_image_dir}/{hr_image_file[:-4]}_with_bounding_box.jpg")
        print('Done')

FileNotFoundError: [Errno 2] No such file or directory: 'benchmark/pcb/LR'

In [10]:
# for (sub, model) in zip(['VDSR', 'SRCNN'], [vdsr, srcnn]):
sub = 'SRResNet'
model = srresnet
lr_image_dir = f'test1_600x600/images'
hr_image_dir = f'test1_600x600/images'
output_image_dir = f'output/{sub}/images'
os.makedirs(output_image_dir, exist_ok = True)
# Duyệt qua các ảnh trong thư mục
lr_image_files = os.listdir(lr_image_dir)
hr_image_files = os.listdir(hr_image_dir)
lr_image_files.sort()
hr_image_files.sort()

psnr_dict = {
    "mouse_bite" :[],
    "spur_":[], 
    "missing_hole":[],
    "short":[],
    "open_circuit":[],
    "spurious_copper":[]

}
ssim_dict = {
    "mouse_bite" :[],
    "spur_":[], 
    "missing_hole":[],
    "short":[],
    "open_circuit":[],
    "spurious_copper":[]

}

start = time.time()
transform = transforms.ToTensor()
with torch.no_grad():
    for lr_image_file, hr_image_file in tqdm(zip(lr_image_files, hr_image_files), unit = 'img'):
        # Đường dẫn đến ảnh
        lr_image_path = os.path.join(lr_image_dir, lr_image_file)
        hr_image_path = os.path.join(hr_image_dir, hr_image_file)
        output_image_path = os.path.join(output_image_dir, lr_image_file)

        # Tải và chuyển đổi ảnh
        lr_image = Image.open(lr_image_path)
        hr_image = Image.open(hr_image_path)
        lr_image = lr_image.resize((150, 150))
        # lr_image = lr_image.resize((600, 600))
        lr_image = transform(lr_image).unsqueeze(0).to(device)  # Thêm batch dimension và chuyển sang CPU
        hr_image = transform(hr_image).unsqueeze(0).to(device)  # Thêm batch dimension và chuyển sang CPU

        # Dự đoán
        output = model(lr_image)

        # Tính toán PSNR
        psnr,ssim = calculate_metrics(output, hr_image)
        for key in psnr_dict.keys():
            if key in lr_image_path:
                psnr_dict[key].append(psnr)
                ssim_dict[key].append(ssim)
                break

        # Chuyển đổi tensor đầu ra thành ảnh và lưu
        output_image = output.squeeze(0).to(device)  # Loại bỏ batch dimension và chuyển tensor sang CPU
        output_image = transforms.ToPILImage()(output_image)  # Chuyển tensor thành ảnh PIL
        output_image.save(output_image_path)  # Lưu ảnh
avg_psnr = [0, 0, 0, 0, 0, 0]
avg_ssim = [0, 0, 0, 0, 0, 0]


# Tính toán PSNR trung bình
avg_psnr[2] = sum(psnr_dict['missing_hole'])/len(psnr_dict['missing_hole']) #missinghole_psnr 
avg_psnr[0] = sum(psnr_dict['mouse_bite'])/len(psnr_dict['mouse_bite']) #mousebite_psnr
avg_psnr[4] = sum(psnr_dict['open_circuit'])/len(psnr_dict['open_circuit']) #opencircuit_psnr
avg_psnr[3] = sum(psnr_dict['short'])/len(psnr_dict['short']) #short_psnr
avg_psnr[1] = sum(psnr_dict['spur_'])/len(psnr_dict['spur_']) #spur_psnr
avg_psnr[5]= sum(psnr_dict['spurious_copper'])/len(psnr_dict['spurious_copper']) #spuriouscopper_psnr 
average_psnr = sum(avg_psnr)/len(avg_psnr)
avg_ssim[2] = sum(ssim_dict['missing_hole'])/len(ssim_dict['missing_hole']) #missinghole_ssim 
avg_ssim[0] = sum(ssim_dict['mouse_bite'])/len(ssim_dict['mouse_bite']) #mousebite_ssim
avg_ssim[4] = sum(ssim_dict['open_circuit'])/len(ssim_dict['open_circuit']) #opencircuit_ssim
avg_ssim[3] = sum(ssim_dict['short'])/len(ssim_dict['short']) #short_ssim
avg_ssim[1] = sum(ssim_dict['spur_'])/len(ssim_dict['spur_']) #spur_ssim
avg_ssim[5]= sum(ssim_dict['spurious_copper'])/len(ssim_dict['spurious_copper']) #spuriouscopper_ssim  
average_ssim = sum(avg_ssim)/len(avg_ssim)
end = time.time()

with open('results.txt', 'a') as f:
    f.write(output_image_dir.split('/')[1] + '\n')
    f.write(f'missinghole_psnr: {avg_psnr[2]:.2f}' + '\n')
    f.write(f'mousebite_psnr: {avg_psnr[0]:.2f}' + '\n')
    f.write(f'opencircuit_psnr: {avg_psnr[4]:.2f}' + '\n')
    f.write(f'short_psnr: {avg_psnr[3]:.2f}' + '\n')
    f.write(f'spur_psnr: {avg_psnr[1]:.2f}' + '\n')
    f.write(f'spuriouscopper_psnr: {avg_psnr[5]:.2f}' + '\n')
    f.write(f'average_psnr: {average_psnr:.2f}' + '\n')
    f.write(f'time process: {end - start:.2f}' + '\n')

    f.write(output_image_dir.split('/')[1] + '\n')
    f.write(f'missinghole_psnr: {avg_ssim[2]:.2f}' + '\n')
    f.write(f'mousebite_psnr: {avg_ssim[0]:.2f}' + '\n')
    f.write(f'opencircuit_psnr: {avg_ssim[4]:.2f}' + '\n')
    f.write(f'short_psnr: {avg_ssim[3]:.2f}' + '\n')
    f.write(f'spur_psnr: {avg_ssim[1]:.2f}' + '\n')
    f.write(f'spuriouscopper_psnr: {avg_ssim[5]:.2f}' + '\n')
    f.write(f'average_psnr: {average_ssim:.2f}' + '\n')
    f.write(f'time process: {end - start:.2f}' + '\n')


# tạo dữ liệu cho YOLO
print(output_image_dir.split('/')[1] + '\n')
print(f'missinghole_psnr: {avg_psnr[2]:.2f}' + '\n')
print(f'mousebite_psnr: {avg_psnr[0]:.2f}' + '\n')
print(f'opencircuit_psnr: {avg_psnr[4]:.2f}' + '\n')
print(f'short_psnr: {avg_psnr[3]:.2f}' + '\n')
print(f'spur_psnr: {avg_psnr[1]:.2f}' + '\n')
print(f'spuriouscopper_psnr: {avg_psnr[5]:.2f}' + '\n')
print(f'average_psnr: {average_psnr:.2f}' + '\n')
print(f'time process: {end - start:.2f}' + '\n')
print(f'missinghole_ssim: {avg_ssim[2]:.2f}' + '\n')
print(f'mousebite_ssim: {avg_ssim[0]:.2f}' + '\n')
print(f'opencircuit_ssim: {avg_ssim[4]:.2f}' + '\n')
print(f'short_ssim: {avg_ssim[3]:.2f}' + '\n')
print(f'spur_ssim: {avg_ssim[1]:.2f}' + '\n')
print(f'spuriouscopper_ssim: {avg_ssim[5]:.2f}' + '\n')
print(f'average_ssim: {average_ssim:.2f}' + '\n')
print(f'time process: {end - start:.2f}' + '\n')
# Đường dẫn đến thư mục nguồn và đích
source_dir = f'test1_600x600/labels'

dest = f'output/{sub}/labels'

# Tạo thư mục đích nếu chưa tồn tại
os.makedirs(dest, exist_ok=True)

# Sao chép các file từ thư mục nguồn sang thư mục đích
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    file1 = os.path.join(dest, filename)

    shutil.copy(source_file, file1)

print("Đã sao chép các file thành công.")


0img [00:00, ?img/s]/home/robot/anaconda3/envs/pcb/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(
/home/robot/anaconda3/envs/pcb/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(
1068img [05:36,  3.17img/s]

SRResNet

missinghole_psnr: 26.71

mousebite_psnr: 26.78

opencircuit_psnr: 26.83

short_psnr: 26.72

spur_psnr: 26.88

spuriouscopper_psnr: 26.64

average_psnr: 26.76

time process: 336.76

missinghole_ssim: 0.75

mousebite_ssim: 0.75

opencircuit_ssim: 0.75

short_ssim: 0.75

spur_ssim: 0.76

spuriouscopper_ssim: 0.75

average_ssim: 0.75

time process: 336.76

Đã sao chép các file thành công.


In [11]:
import yaml

# Nội dung của file YAML
# sub = 'VDSR'
data = {
    'train': f'/home/robot/Desktop/pcb/test1_600x600',
    'val': f'./{sub}/images',
    'nc': 6,
    'names': {
        0: 'mouse_bite',
        1: 'spur',
        2: 'missing_hole',
        3: 'short',
        4: 'open_circuit',
        5: 'spurious_copper'
    }
}

# Tạo và ghi file YAML
with open('output/data.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("File config.yaml đã được tạo và ghi thành công.")


import ultralytics
from ultralytics import YOLO

# Load a model
# model = YOLO("yolov8n.pt")  # load an official model
model = YOLO("bestweight_2006.pt")  # load a custom model

# Validate the model
metrics = model.val(data = 'output/data.yaml', batch = 1, imgsz = 600)  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

File config.yaml đã được tạo và ghi thành công.
WARNING ⚠️ imgsz=[600] must be multiple of max stride 32, updating to [608]
Ultralytics 8.3.9 🚀 Python-3.12.4 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 10971MiB)
Model summary (fused): 168 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/robot/Desktop/pcb/output/SRResNet/labels... 1068 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1068/1068 [00:00<00:00, 2440.41it/s]

val: New cache created: /home/robot/Desktop/pcb/output/SRResNet/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1068/1068 [00:05<00:00, 198.22it/s]


                   all       1068       2158      0.914      0.759      0.837      0.517
            mouse_bite        168        332      0.894      0.762      0.811      0.488
                  spur        169        348      0.739      0.787      0.789      0.491
          missing_hole        190        379      0.987      0.842      0.932      0.602
                 short        184        366      0.967       0.73      0.845      0.541
          open_circuit        166        345      0.963       0.84      0.911      0.538
       spurious_copper        191        388      0.932      0.593      0.737      0.443
Speed: 0.5ms preprocess, 2.8ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val38


array([    0.48829,     0.49108,     0.60177,     0.54134,     0.53768,     0.44313])